In [1]:
import librosa
import numpy as np
from IPython.display import Audio
import torch

In [2]:
from funasr import AutoModel

model = AutoModel(model="iic/emotion2vec_plus_large")

2024-06-06 16:55:43,205 - modelscope - INFO - PyTorch version 2.3.1 Found.
2024-06-06 16:55:43,210 - modelscope - INFO - Loading ast index from /home/mila/s/soroush.omranpour/.cache/modelscope/ast_indexer
2024-06-06 16:55:43,458 - modelscope - INFO - Loading done! Current index file version is 1.15.0, with md5 5eb56b46a2b5de5a5690e9f19ec1ca5d and a total number of 980 components indexed
2024-06-06 16:55:45,143 - modelscope - WARNING - Using the master branch is fragile, please use it with caution!
2024-06-06 16:55:45,146 - modelscope - INFO - Use user-specified model revision: master
Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1.81G/1.81G [01:50<00:00, 17.5MB/s]
Downloading: 100%|██████████████████████████

Detect model requirements, begin to install it: /home/mila/s/soroush.omranpour/.cache/modelscope/hub/iic/emotion2vec_plus_large/requirements.txt
install model requirements successfully
ckpt: /home/mila/s/soroush.omranpour/.cache/modelscope/hub/iic/emotion2vec_plus_large/model.pt
init param, map: modality_encoders.AUDIO.extra_tokens from d2v_model.modality_encoders.AUDIO.extra_tokens in ckpt
init param, map: modality_encoders.AUDIO.alibi_scale from d2v_model.modality_encoders.AUDIO.alibi_scale in ckpt
init param, map: modality_encoders.AUDIO.local_encoder.conv_layers.0.0.weight from d2v_model.modality_encoders.AUDIO.local_encoder.conv_layers.0.0.weight in ckpt
init param, map: modality_encoders.AUDIO.local_encoder.conv_layers.0.2.1.weight from d2v_model.modality_encoders.AUDIO.local_encoder.conv_layers.0.2.1.weight in ckpt
init param, map: modality_encoders.AUDIO.local_encoder.conv_layers.0.2.1.bias from d2v_model.modality_encoders.AUDIO.local_encoder.conv_layers.0.2.1.bias in ckpt
init

In [16]:
model.model

Emotion2vec(
  (modality_encoders): ModuleDict(
    (AUDIO): AudioEncoder(
      (local_encoder): ConvFeatureExtractionModel(
        (conv_layers): ModuleList(
          (0): Sequential(
            (0): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
            (1): Dropout(p=0.0, inplace=False)
            (2): Sequential(
              (0): TransposeLast()
              (1): Fp32LayerNorm((512,), eps=1e-05, elementwise_affine=True)
              (2): TransposeLast()
            )
            (3): GELU(approximate='none')
          )
          (1-4): 4 x Sequential(
            (0): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
            (1): Dropout(p=0.0, inplace=False)
            (2): Sequential(
              (0): TransposeLast()
              (1): Fp32LayerNorm((512,), eps=1e-05, elementwise_affine=True)
              (2): TransposeLast()
            )
            (3): GELU(approximate='none')
          )
          (5-6): 2 x Sequential(
           

In [3]:
y, fs = librosa.load('LANDR- Meditation- Transcend 15 Mins-Balanced-Medium.mp3', sr=16000)
y.shape, fs

((14401202,), 16000)

In [12]:
import math
from tqdm.notebook import tqdm

classes = ['angry', 'happy', 'neutral', 'sad', 'unknown']

cache = {}
n = y.shape[0]
m = 20 * fs
k = math.ceil(n / m)

res = []
for i in tqdm(range(k)):
    scores = model.generate(
        input=y[i*m:(i+1)*m], 
        is_final=(i == (k - 1)), 
        chunk_size=m,
        cache=cache
    )[0]['scores']
    res += [classes[np.argmax(scores)]]

  0%|          | 0/46 [00:00<?, ?it/s]


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:12<00:00, 12.50s/it]
{'load_data': '0.000', 'extract_feat': 0.0, 'forward': '12.496', 'batch_size': '1', 'rtf': '0.625'}, : 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:12<00:00, 12.50s/it]
rtf_avg: 0.625: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

In [14]:
res

['sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'neutral']